# Tutorial 2 - RNN Time Series

In this notebook, we will predict the weather temperature. 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error


# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)



# Read the Dataset

In [ ]:
import pandas as pd

weather = pd.read_csv('weather.csv')

weather.head()

In [ ]:
# Convert the temp to Fahrenheit:

weather['Air Temp F'] = weather['Air Temp.']*1.8 + 32

In [ ]:
weather

In [ ]:
#Drop the columns we don't need

weather.drop(['NO2', 'CO', 'O3', 'NO', 'PM2.5', 'PM10', 'Air Temp.',
              'Air Hum.', 'Air Pres.'], axis=1, inplace=True )

In [ ]:
weather

In [ ]:
#Plot temp

plt.plot(weather['Air Temp F'])
plt.show()

# Data Cleanup

In [ ]:
# Values higher than 100 degrees are probably incorrect readings

weather[weather['Air Temp F']>100]

In [ ]:
# Convert all values higher than 100 degrees to null values

weather['Air Temp F'] = np.where(weather['Air Temp F']>100, np.nan, weather['Air Temp F'])

In [ ]:
# Values lower than 30 degrees are probably incorrect readings. Convert them to null

weather['Air Temp F'] = np.where(weather['Air Temp F']<30, np.nan, weather['Air Temp F'])

In [ ]:
plt.plot(weather['Air Temp F'])
plt.show()

In [ ]:
# Remove all null values

weather.dropna(inplace=True)

weather.reset_index(drop=True, inplace=True)

In [ ]:
plt.plot(weather['Air Temp F'])
plt.show()

## RESHAPE the data set!

In [ ]:
weather.shape

In [ ]:
# Note that not all days have 24 readings. Some are missing.

weather.shape[0]/24

In [ ]:
# Find the reading count for each day

hour_count = pd.DataFrame(weather.groupby(['date']).count()['hour'])

hour_count

In [ ]:
# Find the reading counts that are less than 24

hour_count[hour_count['hour']<24]

In [ ]:
# Identify the dates of these records

hour_count[hour_count['hour']<24].index.values

In [ ]:
# Find the corresponding index values in the original data set

indexes = weather[weather['date'].isin(hour_count[hour_count['hour']<24].index.values)]

indexes

In [ ]:
# Remove these rows from the data set.

weather.drop(indexes.index, axis=0, inplace=True)

weather.reset_index(drop=True)

weather.shape

In [ ]:
weather

In [ ]:
# All remaining days have 24 readings (for 24 hours)
# There are a total of 288 days

weather.shape[0]/24

In [ ]:
plt.plot(weather['Air Temp F'])
plt.show()

In [ ]:
# Re-organize the data set by day and hours

temp = np.array(weather['Air Temp F']).reshape(288,24)

temp

In [ ]:
# Convert to dataframe

temp_df = pd.DataFrame(temp, columns=np.arange(0,24,1))

temp_df

# Reshape for Standardizing Data

In [ ]:
# Let's create a single sequence (i.e., feature) for standardization

temp_1feature = np.array(temp_df).ravel().reshape(-1,1)

temp_1feature.shape

In [ ]:
temp_1feature

## Standardize the values

In [ ]:
# Next, standardize

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

temp_std = scaler.fit_transform(temp_1feature)

## Reshape the data back to 24-hour format

In [ ]:
temp_reshaped = temp_std.reshape(288,24)

temp_reshaped.shape

In [ ]:
#Pandas version of the reshaped data

pd.DataFrame(temp_reshaped, columns=np.arange(0,24,1))

# Split the Data

**We CANNOT do random train/test split**

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(temp_reshaped, test_size=0.3)

In [ ]:
train.shape

In [ ]:
test.shape

# Create Input and Target values

The first 23 hours will be input to predict the 24th hour reading (i.e., target)

In [ ]:
# The first 23 columns (from 0 to 22) are inputs

train_inputs = train[:,:23]


pd.DataFrame(train_inputs, columns=np.arange(0,23,1))

## Add one more dimension to make it ready for RNNs

In [ ]:
#Create an additional dimension for train

train_x = train_inputs[...,np.newaxis]

train_x.shape 

## Set the target

In [ ]:
# The last column (23) is TARGET

train_target = train[:,-1]


pd.DataFrame(train_target, columns=['23'])

## Repeat for TEST

In [ ]:
test.shape

In [ ]:
# The first 23 columns (from 0 to 22) are inputs

test_inputs = test[:,:23]

In [ ]:
#Create an additional dimension for test

test_x = test_inputs[...,np.newaxis]

test_x.shape 

In [ ]:
# The last column (23) is TARGET

test_target = test[:,-1]


pd.DataFrame(test_target, columns=['23'])

# A normal (cross-sectional) NN

This model assumes that the data is NOT a time-series data set. It treats the data as cross-sectional and the columns being independent of each other.

In [ ]:
model = keras.models.Sequential([
    
    keras.layers.Flatten(input_shape=[23, 1]),
    keras.layers.Dense(23, activation='relu'),
    keras.layers.Dense(1, activation=None)
    
])

model.compile(loss="mse", optimizer='Adam')

history = model.fit(train_x, train_target, epochs=100)

### Predictions

In [ ]:
#Predict:
y_pred = model.predict(test_x)

In [ ]:
# Remember, these are standardized values. 

comparison = pd.DataFrame()

comparison['actual'] = scaler.inverse_transform([test_target]).flatten()
comparison['predicted'] = scaler.inverse_transform(y_pred).flatten()

In [ ]:
comparison

In [ ]:
mean_squared_error(comparison['actual'], comparison['predicted'])

In [ ]:
plt.plot(comparison['actual'], label = 'actual')
plt.plot(comparison['predicted'], label = 'predicted')

plt.legend()

plt.show()

# Simple RNN

Simplest recurrent neural network

In [ ]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(32, activation='relu', input_shape=[23, 1]),
    keras.layers.Dense(1, activation=None)
])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model.compile(loss="mse", optimizer='Adam')

history = model.fit(train_x, train_target, epochs=30)

### Predictions

In [ ]:
#Predict:
y_pred = model.predict(test_x)

In [ ]:
#Remember, these are standardized values. 

comparison = pd.DataFrame()

comparison['actual'] = scaler.inverse_transform([test_target]).flatten()
comparison['predicted'] = scaler.inverse_transform(y_pred).flatten()

In [ ]:
comparison

In [ ]:
mean_squared_error(comparison['actual'], comparison['predicted'])

In [ ]:
plt.plot(comparison['actual'], label = 'actual')
plt.plot(comparison['predicted'], label = 'predicted')

plt.legend()

plt.show()

## Simple RNN with more layers

**Be careful: when stacking RNN layers, you have to set "return_sequences" to True. This enables the layer to send a "sequence" of values to the next layer -- jut like how it uses a sequence of values for training. However, if the output of RNN is sent to a DENSE layer, then a single value should be sent. That's why there is no "return sequences" right before DENSE layers.**

In [ ]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(32, activation='relu', return_sequences=True, input_shape=[23, 1]),
    keras.layers.SimpleRNN(32, activation='relu', return_sequences=False),
    keras.layers.Dense(1, activation=None)
])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model.compile(loss="mse", optimizer='Adam')

history = model.fit(train_x, train_target, epochs=30)

### Predictions

In [ ]:
#Predict:
y_pred = model.predict(test_x)


In [ ]:
#Remember, these are standardized values. 

comparison = pd.DataFrame()

comparison['actual'] = scaler.inverse_transform([test_target]).flatten()
comparison['predicted'] = scaler.inverse_transform(y_pred).flatten()

In [ ]:
comparison

In [ ]:
mean_squared_error(comparison['actual'], comparison['predicted'])

In [ ]:
plt.plot(comparison['actual'], label = 'actual')
plt.plot(comparison['predicted'], label = 'predicted')

plt.legend()

plt.show()

# LSTM with one layer

In [ ]:
model = keras.models.Sequential([
    keras.layers.LSTM(32, activation='relu', input_shape=[23, 1]),
    keras.layers.Dense(1, activation=None)
])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model.compile(loss="mse", optimizer='Adam')

history = model.fit(train_x, train_target, epochs=30)

### Predictions

In [ ]:
#Predict:
y_pred = model.predict(test_x)


In [ ]:
#Remember, these are standardized values. 

comparison = pd.DataFrame()

comparison['actual'] = scaler.inverse_transform([test_target]).flatten()
comparison['predicted'] = scaler.inverse_transform(y_pred).flatten()

In [ ]:
mean_squared_error(comparison['actual'], comparison['predicted'])

In [ ]:
plt.plot(comparison['actual'], label = 'actual')
plt.plot(comparison['predicted'], label = 'predicted')

plt.legend()

plt.show()

# LSTM with more layers

In [ ]:
model = keras.models.Sequential([
    keras.layers.LSTM(32, activation='tanh', return_sequences=True, input_shape=[23, 1]),
    keras.layers.LSTM(32, activation='tanh', return_sequences=False),
    keras.layers.Dense(1, activation=None)
])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model.compile(loss="mse", optimizer='Adam')

history = model.fit(train_x, train_target, epochs=30)

### Predictions

In [ ]:
#Predict:
y_pred = model.predict(test_x)

In [ ]:
#Remember, these are standardized values. 

comparison = pd.DataFrame()

comparison['actual'] = scaler.inverse_transform([test_target]).flatten()
comparison['predicted'] = scaler.inverse_transform(y_pred).flatten()

In [ ]:
mean_squared_error(comparison['actual'], comparison['predicted'])

In [ ]:
plt.plot(comparison['actual'], label = 'actual')
plt.plot(comparison['predicted'], label = 'predicted')

plt.legend()

plt.show()

# GRU (with more layers)

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(32, activation='relu', return_sequences=True, input_shape=[23, 1]),
    keras.layers.GRU(32, activation='relu', return_sequences=False),
    keras.layers.Dense(1, activation=None)
])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model.compile(loss="mse", optimizer='RMSprop')

history = model.fit(train_x, train_target, epochs=30)

### Predictions

In [ ]:
#Predict:
y_pred = model.predict(test_x)

In [ ]:
#Remember, these are standardized values. 

comparison = pd.DataFrame()

comparison['actual'] = scaler.inverse_transform([test_target]).flatten()
comparison['predicted'] = scaler.inverse_transform(y_pred).flatten()

In [ ]:
mean_squared_error(comparison['actual'], comparison['predicted'])

In [ ]:
plt.plot(comparison['actual'], label = 'actual')
plt.plot(comparison['predicted'], label = 'predicted')

plt.legend()

plt.show()

# Conv1D

### Last Layer: GRU (you can change it to SimpleRNN or LSTM as well)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=3, strides=1, padding="valid", input_shape=[23, 1]),
    keras.layers.GRU(32, activation='relu', return_sequences=True),
    keras.layers.GRU(32, activation='relu', return_sequences=False),
    keras.layers.Dense(1, activation=None)
])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model.compile(loss="mse", optimizer='Adam')

history = model.fit(train_x, train_target, epochs=30)

### Predictions

In [ ]:
#Predict:
y_pred = model.predict(test_x)

In [ ]:
#Remember, these are standardized values. 

comparison = pd.DataFrame()

comparison['actual'] = scaler.inverse_transform([test_target]).flatten()
comparison['predicted'] = scaler.inverse_transform(y_pred).flatten()

In [ ]:
mean_squared_error(comparison['actual'], comparison['predicted'])

# Forecasting Several Steps Ahead

## Now let's create an RNN that predicts 12 next values at once:

In [ ]:
# The first 12 columns (from 0 to 11) are inputs

train_inputs = train[:,:12]

pd.DataFrame(train_inputs, columns=np.arange(0,12,1))

In [ ]:
#Create an additional dimension for train

train_x = train_inputs.reshape(201,12,1)

train_x.shape 

In [ ]:
# The last 12 readings (from 12 to 23) are TARGET

train_target = train[:,-12:]

pd.DataFrame(train_target, columns=np.arange(12,24,1))

## Repeat for TEST

In [ ]:
# The first 12 columns (from 0 to 11) are inputs

test_inputs = test[:,:12]

pd.DataFrame(test_inputs, columns=np.arange(0,12,1))

In [ ]:
#Create an additional dimension for test

test_x = test_inputs.reshape(87,12,1)

test_x.shape 

In [ ]:
# The last 12 columns are TARGET

test_target = test[:,-12:]

pd.DataFrame(test_target, columns=np.arange(12,24,1))

# GRU

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(32, activation='relu', return_sequences=True, input_shape=[12, 1]),
    keras.layers.GRU(32, activation='relu', return_sequences=False),
    keras.layers.Dense(12, activation=None)
])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model.compile(loss="mse", optimizer='Adam')

history = model.fit(train_x, train_target, epochs=30)

### Predictions

In [ ]:
#Predict:
y_pred = model.predict(test_x)

In [ ]:
#Remember, these are standardized values. 

actual = pd.DataFrame(scaler.inverse_transform(test_target))
predicted = pd.DataFrame(scaler.inverse_transform(y_pred))

In [ ]:
actual

In [ ]:
predicted

In [ ]:
mean_squared_error(actual, predicted)

In [ ]:
# Plot a random row to see the accuracy of predictions

random_row = np.random.randint(low=0, high=86)

plt.plot(actual.iloc[random_row], label='actual')
plt.plot(predicted.iloc[random_row], label='predicted')

plt.legend()
plt.show()

# Sliding window

Prior 18 hours predicts next 6 hours

In [ ]:
steps_for_prediction = 18
steps_to_predict = 6

#Be careful: sums to 24 hours

In [ ]:
train

In [ ]:
train.flatten().shape

In [ ]:
train_inputs_sw = []
train_target_sw = []

for i in range(0,4824-24):
    input_row = train.flatten()[i:i+steps_for_prediction]
    target_row = train.flatten()[i+steps_for_prediction:i+steps_for_prediction+steps_to_predict]
    train_inputs_sw.append((input_row))
    train_target_sw.append((target_row))

In [ ]:
train_inputs = np.vstack(train_inputs_sw)

train_targets = np.vstack(train_target_sw)

In [ ]:
train_targets.shape

In [ ]:
# Repeat for test

test_inputs_sw = []
test_target_sw = []

for i in range(0,test.flatten().shape[0]-24):
    input_row = test.flatten()[i:i+steps_for_prediction]
    target_row = test.flatten()[i+steps_for_prediction:i+steps_for_prediction+steps_to_predict]
    test_inputs_sw.append((input_row))
    test_target_sw.append((target_row))
    
test_inputs = np.vstack(test_inputs_sw)

test_targets = np.vstack(test_target_sw)

# GRU

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(32, activation='relu', return_sequences=True, input_shape=[18, 1]),
    keras.layers.GRU(32, activation='relu', return_sequences=False),
    keras.layers.Dense(steps_to_predict, activation=None)
])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model.compile(loss="mse", optimizer='Adam')

history = model.fit(train_inputs, train_targets, epochs=30)

### Predictions

In [ ]:
#Predict:
y_pred = model.predict(test_inputs)

In [ ]:
#Remember, these are standardized values. 

actual = pd.DataFrame(scaler.inverse_transform(test_targets))
predicted = pd.DataFrame(scaler.inverse_transform(y_pred))

In [ ]:
actual

In [ ]:
predicted

In [ ]:
mean_squared_error(actual, predicted)

In [ ]:
# Plot a random row to see the accuracy of predictions

random_row = np.random.randint(low=0, high=2063)

plt.plot(actual.iloc[random_row], label='actual')
plt.plot(predicted.iloc[random_row], label='predicted')

plt.legend()
plt.show()

## We could try using 6 steps to predict the next 6 steps (maybe 18 steps is too long)